In [27]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from tensorflow import keras
from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트
%matplotlib inline

In [28]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [29]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")
dog_train=pd.read_csv("trainDoginfo.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

dog_data
# print(dog_train)
# print(kindCd)

,kindNum,neuterYn,sexCd,weight,noticeDays,age2,processState
0,128.0,N,F,7.46,10,12,0
1,114.0,N,M,7.00,14,1,1
2,114.0,U,M,4.50,11,2,0
3,67.0,N,M,10.00,8,1,0
4,114.0,N,M,6.00,8,4,0
...,...,...,...,...,...,...,...
22787,114.0,N,M,1.00,10,0,1
22788,114.0,N,M,1.00,10,0,1
22789,114.0,N,M,1.00,10,0,1
22790,128.0,U,F,6.00,12,3,0


In [30]:
kindCd = np.array(kindCd_data, dtype = np.float64)
kindCd_train=np.array(kindCd_data, dtype=np.float64)

kindCd = kindCd.reshape(177)
kindCd_train=kindCd_train.reshape(177)
print(kindCd)
print(kindCd.shape)

[ 54.  56.  55. 118. 115.  37.  81. 204.  83.  82.  38.  39.  40.  43.
  42. 153.  41. 120. 155.  69.  71. 142.  93. 167.  70. 166.  94. 121.
 152.  73. 146.  72. 159.  76.  75.  79.  78.  77.  74.  80. 114. 133.
  12.  17.  15. 164. 157. 148.  16.  20.  21.  22.  24. 208.  23.  26.
  27. 169.  25.  19.  13.  18.  14. 162.  85.  96.  95.   1.  34. 104.
  31.  99. 122. 123.  97. 132. 105. 154. 124. 100. 103. 151. 139. 101.
 102.  98. 136. 202. 160. 203.   8. 131.   9. 119. 150. 210.  57.  58.
  59.   6.   4.   7.   5. 143.  11.  10. 137.  84. 163. 112. 113. 149.
 211. 110. 205. 108. 109.  60.  46.  47.  44.  45.  53.  62.  61.  52.
 165.  51. 156. 129.  67.  35.  33.  32. 158. 144.  30.  29.  64. 207.
  28.   2.  68. 125. 141. 145.  36.  66.  65.  63. 140. 107. 106. 209.
  86.  88.  90.  87. 138.  89. 126. 127. 128.  91.   3. 161.  50. 168.
  49. 147.  92.  48. 135. 206. 130. 134. 111.]
(177,)


In [31]:
# kindNum을 원핫 인코딩
kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# kindCd

# 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_data.drop(['kindNum'], axis='columns', inplace=True)
dog_data = pd.concat([dog_data, kindCd], axis=1)


# kindNum을 원핫 인코딩
kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_train.drop(['kindNum'], axis='columns', inplace=True)
dog_train = pd.concat([dog_train, kindCd_train], axis=1)
dog_data
# dog_train

,neuterYn,sexCd,weight,noticeDays,age2,processState,1.0,2.0,3.0,4.0,...,202.0,203.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,211.0
0,N,F,7.46,10,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,N,M,7.00,14,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,U,M,4.50,11,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,N,M,10.00,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,N,M,6.00,8,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22787,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22788,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22789,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22790,U,F,6.00,12,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [33]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)
data.head()
train_data.head()
# print(type(dog_data))
# print(type(data))

print(data)

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)

# data
# print(data.shape)
# print(len(train_data))
# print(train_data.shape)

       weight  noticeDays  age2  processState  1.0  2.0  3.0  4.0  5.0  6.0  \
0        7.46          10    12             0    0    0    0    0    0    0   
1        7.00          14     1             1    0    0    0    0    0    0   
2        4.50          11     2             0    0    0    0    0    0    0   
3       10.00           8     1             0    0    0    0    0    0    0   
4        6.00           8     4             0    0    0    0    0    0    0   
...       ...         ...   ...           ...  ...  ...  ...  ...  ...  ...   
22787    1.00          10     0             1    0    0    0    0    0    0   
22788    1.00          10     0             1    0    0    0    0    0    0   
22789    1.00          10     0             1    0    0    0    0    0    0   
22790    6.00          12     3             0    0    0    0    0    0    0   
22791    3.50          10     0             0    0    0    0    0    0    0   

       ...  208.0  209.0  210.0  211.0  neuterYn_N 

In [34]:
a = data[:, :3]
b = data[:, 4:]

# print(a[0,:])
# print(b)

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape

[[ 7.46 10.   12.   ...  1.    0.    0.  ]
 [ 7.   14.    1.   ...  0.    1.    0.  ]
 [ 4.5  11.    2.   ...  0.    1.    0.  ]
 ...
 [ 1.   10.    0.   ...  0.    1.    0.  ]
 [ 6.   12.    3.   ...  1.    0.    0.  ]
 [ 3.5  10.    0.   ...  0.    1.    0.  ]]


(22777, 186)

In [35]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
# yData = yData.astype('int32')
print(yData)
type(yData)
yData.dtype

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [36]:
# model=Sequential()
# model.add(Dense(1, input_dim=186, activation='sigmoid'))
# sgd=optimizers.SGD(lr=0.01)
# model.compile(optimizer=sgd ,loss='binary_crossentropy',metrics=['binary_accuracy'])
# # 옵티마이저는 경사하강법 sgd를 사용합니다.
# # 손실 함수(Loss function)는 binary_crossentropy(이진 크로스 엔트로피)를 사용합니다.
# model.fit(xData,yData, batch_size=128, epochs=200, shuffle=False)
# # 주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 200번 시도합니다.

In [109]:
# l2_model = keras.models.Sequential([
#     keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu, input_shape=(186,)),
#     keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu),
#     keras.layers.Dense(1, activation=tf.nn.sigmoid)
# ])

from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Dropout
from keras.optimizers import Adadelta


def mlp_model():
    model = Sequential()

#     model = Sequential([
#     Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu, input_shape=(186,)),
#     Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu),
#     Dense(1, activation=tf.nn.sigmoid)
# ])

    model.add(Dense(16, input_shape = (186, )))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Activation('relu'))
#     model.add(Dense(50, input_shape = (186, )))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
    model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
    model.add(Dense(1))
    model.add(Activation('sigmoid'))   
#     model.add(Dropout(0.5))
#     model.add(Dense(1))
#     model.add(Activation('softmax'))

#     sgd=optimizers.SGD(lr=0.01, momentum=0.9)

    model.compile(optimizer=Adadelta(rho=0.95),
                     loss='binary_crossentropy',
                     metrics=['accuracy', 'binary_crossentropy'])
    return model

# mlp_model(l2_model)

# sgd=optimizers.SGD(lr=0.01)

# l2_model.compile(optimizer='sgd',
#                  loss='binary_crossentropy',
#                  metrics=['accuracy', 'binary_crossentropy'])

l2_model = mlp_model()

l2_model_history = l2_model.fit(xData, yData,
                                epochs=100,
                                shuffle = True,
                                batch_size=32,
                                validation_data=(xData, yData),
                                verbose=1)

l2_model.summary()

Train on 22777 samples, validate on 22777 samples
Epoch 1/100
22777/22777 [==============================] - 11s 489us/step - loss: 0.6289 - accuracy: 0.6491 - binary_crossentropy: 0.6289 - val_loss: 0.6172 - val_accuracy: 0.6692 - val_binary_crossentropy: 0.6172
Epoch 2/100
22777/22777 [==============================] - 14s 627us/step - loss: 0.6050 - accuracy: 0.6759 - binary_crossentropy: 0.6050 - val_loss: 0.5990 - val_accuracy: 0.6860 - val_binary_crossentropy: 0.5990
Epoch 3/100
22777/22777 [==============================] - 14s 595us/step - loss: 0.6008 - accuracy: 0.6773 - binary_crossentropy: 0.6008 - val_loss: 0.6009 - val_accuracy: 0.6852 - val_binary_crossentropy: 0.6009
Epoch 4/100
22777/22777 [==============================] - 12s 526us/step - loss: 0.5986 - accuracy: 0.6814 - binary_crossentropy: 0.5986 - val_loss: 0.6003 - val_accuracy: 0.6805 - val_binary_crossentropy: 0.6003
Epoch 5/100
22777/22777 [==============================] - 12s 524us/step - loss: 0.5955 - acc

22777/22777 [==============================] - 12s 547us/step - loss: 0.5864 - accuracy: 0.6875 - binary_crossentropy: 0.5864 - val_loss: 0.5846 - val_accuracy: 0.6924 - val_binary_crossentropy: 0.5846
Epoch 39/100
22777/22777 [==============================] - 11s 478us/step - loss: 0.5874 - accuracy: 0.6858 - binary_crossentropy: 0.5874 - val_loss: 0.5780 - val_accuracy: 0.6958 - val_binary_crossentropy: 0.5780
Epoch 40/100
22777/22777 [==============================] - 12s 543us/step - loss: 0.5860 - accuracy: 0.6899 - binary_crossentropy: 0.5860 - val_loss: 0.5797 - val_accuracy: 0.6946 - val_binary_crossentropy: 0.5797
Epoch 41/100
22777/22777 [==============================] - 13s 576us/step - loss: 0.5859 - accuracy: 0.6895 - binary_crossentropy: 0.5859 - val_loss: 0.5809 - val_accuracy: 0.6916 - val_binary_crossentropy: 0.5809
Epoch 42/100
22777/22777 [==============================] - 10s 434us/step - loss: 0.5852 - accuracy: 0.6925 - binary_crossentropy: 0.5852 - val_loss: 0.

22777/22777 [==============================] - 10s 433us/step - loss: 0.5830 - accuracy: 0.6918 - binary_crossentropy: 0.5830 - val_loss: 0.5761 - val_accuracy: 0.6969 - val_binary_crossentropy: 0.5761
Epoch 77/100
22777/22777 [==============================] - 11s 499us/step - loss: 0.5834 - accuracy: 0.6906 - binary_crossentropy: 0.5834 - val_loss: 0.5753 - val_accuracy: 0.6979 - val_binary_crossentropy: 0.5753
Epoch 78/100
22777/22777 [==============================] - 11s 491us/step - loss: 0.5842 - accuracy: 0.6902 - binary_crossentropy: 0.5842 - val_loss: 0.5764 - val_accuracy: 0.6977 - val_binary_crossentropy: 0.5764
Epoch 79/100
22777/22777 [==============================] - 13s 571us/step - loss: 0.5849 - accuracy: 0.6924 - binary_crossentropy: 0.5849 - val_loss: 0.5761 - val_accuracy: 0.6958 - val_binary_crossentropy: 0.5761
Epoch 80/100
22777/22777 [==============================] - 12s 538us/step - loss: 0.5826 - accuracy: 0.6900 - binary_crossentropy: 0.5826 - val_loss: 0.

In [15]:
for i in range(len(train_data)):
    new_x=train_data[i, :].reshape(1,186)
    print('입양 확률 : %8.4f%%' % (l2_model.predict(new_x)*100))
l2_model.save('l2_model')

입양 확률 :  94.2203%
입양 확률 :  36.7390%
입양 확률 :  35.8996%
입양 확률 :  78.0932%
입양 확률 :  21.0893%
입양 확률 :  21.0893%
입양 확률 :  91.3862%
입양 확률 :  78.4416%
입양 확률 :  89.5666%
입양 확률 :  84.8468%
입양 확률 :  72.6395%
입양 확률 :  85.3167%
입양 확률 :  55.5937%
입양 확률 :  49.5257%
입양 확률 :  51.3644%
입양 확률 :  28.9706%
입양 확률 :  44.0449%
입양 확률 :  37.9210%
입양 확률 :  83.8701%
입양 확률 :  23.1563%
입양 확률 :  64.8517%
입양 확률 :  64.8517%
입양 확률 :  53.8228%
입양 확률 :  46.9142%
입양 확률 :  71.4867%
입양 확률 :  58.0059%
입양 확률 :  54.1604%
입양 확률 :  57.8769%
입양 확률 :  41.1052%
입양 확률 :  54.8027%
입양 확률 :  56.1087%
입양 확률 :  41.1052%
입양 확률 :  40.0840%
입양 확률 :  37.2096%
입양 확률 :  90.8167%
입양 확률 :  88.6520%
입양 확률 :  19.2490%
입양 확률 :  69.2019%
입양 확률 :  76.6373%
입양 확률 :  28.4616%
입양 확률 :  26.2082%
입양 확률 :  37.8162%
입양 확률 :  49.3927%
입양 확률 :  31.0500%
입양 확률 :  32.5425%
입양 확률 :  45.5628%
입양 확률 :  49.3927%
입양 확률 :  51.3507%
입양 확률 :  36.3324%
입양 확률 :  45.9654%
입양 확률 :  29.5362%
입양 확률 :  29.1457%
입양 확률 :  55.1988%
입양 확률 :  52.2923%
입양 확률 :  54.0260%
입양 확률 :  5

입양 확률 :  28.6237%
입양 확률 :  93.3353%
입양 확률 :  32.6441%
입양 확률 :  37.7122%
입양 확률 :  67.2223%
입양 확률 :  23.0380%
입양 확률 :  70.3892%
입양 확률 :  77.7126%
입양 확률 :  55.1501%
입양 확률 :  57.7917%
입양 확률 :  67.0752%
입양 확률 :  32.6870%
입양 확률 :  88.6794%
입양 확률 :  50.1714%
입양 확률 :  34.4334%
입양 확률 :  30.2486%
입양 확률 :  57.1593%
입양 확률 :  53.8719%
입양 확률 :  71.0483%
입양 확률 :  66.8481%
입양 확률 :  83.2895%
입양 확률 :  59.9851%
입양 확률 :  87.3223%
입양 확률 :  94.8906%
입양 확률 :  53.2306%
입양 확률 :  41.5467%
입양 확률 :  41.7502%
입양 확률 :  26.5065%
입양 확률 :  29.7671%
입양 확률 :  96.1752%
입양 확률 :  44.4502%
입양 확률 :  92.8657%
입양 확률 :  39.4909%
입양 확률 :  83.0424%
입양 확률 :  91.3181%
입양 확률 :  86.7357%
입양 확률 :  54.5117%
입양 확률 :  23.3083%
입양 확률 :  24.4934%
입양 확률 :  90.5356%
입양 확률 :  65.2899%
입양 확률 :  32.4555%
입양 확률 :  20.8052%
입양 확률 :  31.4452%
입양 확률 :  50.1362%
입양 확률 :  52.8710%
입양 확률 :  53.2796%
입양 확률 :  53.4489%
입양 확률 :  85.6347%
입양 확률 :  89.7685%
입양 확률 :  30.2853%
입양 확률 :  57.8625%
입양 확률 :  33.0917%
입양 확률 :  52.3523%
입양 확률 :  90.5433%
입양 확률 :  9